In [4]:
%run ../../scripts/bios_utils.py

In [1]:
import escher
import modelseed_escher
from modelseed_escher.map import EscherCluster

In [2]:
escher_manager = modelseed_escher.EscherManager(escher)

In [5]:
data = read_json('/Users/fliu/workspace/jupyter/data/www/annotation/data/latest_cluster.json')
em = modelseed_escher.core.EscherMap(data['escher_map'])
em_grid = data['grid']

In [6]:
#em.display_in_notebook()

In [7]:
%run ../escher_grid_merge.py

In [8]:
em_layer = add_layers(em.clone(), em_grid, escher_manager)

150 ModelSEED.Ribitol et al Utilization 0 0 4500 2427 c


In [9]:
#em_layer.display_in_notebook()

In [10]:
ec = EscherCluster(25)
cluster_data = ec.cluster(em_layer)
result = report(cluster_data, em_layer)
result = sort_by_database(result, 'seed.compound')

In [54]:
builder = escher.Builder(map_json=json.dumps(em.escher_map))

In [93]:
import logging
logger = logging.getLogger(__name__)
def report(cluster_data, em):
    cluster_map = em.escher_map
    any_merge = True
    result = {
        'models' : set(),
        'records' : []
    }
    for cluster in cluster_data:
        logger.debug("%s", cluster)
        record = {
            'model_data' : {},
            'database' : {}
        }
        match_cmp = '?'
        cmps = set()
        for id in cluster:
            node_id = cluster_map[1]['nodes'][id]['bigg_id']
            logger.debug("%s", node_id)
            if node_id.startswith('cpd') and '_' in node_id:
                cpd_id, cmp = node_id.split('_')
                cmps.add(cmp)

        if len(cmps) == 1:
            match_cmp = cmps.pop()
        for id in cluster:
            node_id = cluster_map[1]['nodes'][id]['bigg_id']
            cpd_id = node_id
            database = None
            if '@' in node_id:
                cpd_id, database = node_id.split('@')

            #print(cpd_id, database)
            #if is model
            if not database == None:
                result['models'].add(database)
                cmp = match_cmp
                if not cmp in record['model_data']:
                    record['model_data'][cmp] = {}
                if not database in record['model_data'][cmp]:
                    record['model_data'][cmp][database] = set()
                record['model_data'][cmp][database].add(cpd_id)
            else:
                database = 'seed.compound'
                if not database in record['database']:
                    record['database'][database] = set()
                if node_id.startswith('cpd') and '_' in node_id:
                    cpd_id, cmp = node_id.split('_')
                record['database'][database].add(cpd_id)
            #elseif database
        if len(record['model_data']) > 0:
            result['records'].append(record)


    result['models'] = list(result['models'])
    for r in result['records']:
        for cmp_id in r['model_data']:
            for model_id in r['model_data'][cmp_id]:
                r['model_data'][cmp_id][model_id] = list(r['model_data'][cmp_id][model_id])
        for database_id in r['database']:
            r['database'][database_id] = list(r['database'][database_id])
            
    return result
dd = report(cluster_data, em_layer)



In [94]:
dd['records'][0]

{'model_data': {'c': {'iMM904': ['M_nadp_c']}},
 'database': {'seed.compound': ['cpd00006']}}

In [1]:
def sort_by_database(report, db):
    result = {
        'models' : report['models']
    }
    m = {}
    records = []
    for r in report['records']:
        if 'seed.compound' in r['database']:
            key = tuple(sorted(r['database']['seed.compound']))
            if not key in m:
                m[key] = {}
            for cmp in r['model_data']:
                if not cmp in m[key]:
                    m[key][cmp] = {}
                for model_id in r['model_data'][cmp]:
                    if not model_id in m[key][cmp]:
                        m[key][cmp][model_id] = set()
                    m[key][cmp][model_id] |= set(r['model_data'][cmp][model_id])
        else:
            records.append(r)
    for key in m:
        record = {
            'model_data' : {},
            'database' : {db : list(key)}
        }
        for cmp in m[key]:
            record[cmp] = {}
            for model_id in m[key][cmp]:
                record[cmp][model_id] = list(m[key][cmp][model_id])
        records.append(record)
        print(record)
    result['records'] = records
    return report
    
#sort_by_database(dd, 'seed.compound')


In [11]:
import escher
escher_manager = modelseed_escher.EscherManager(escher)

In [92]:
%run ../escher_factory_api.py

!!! cpd00011


In [72]:
em = escher_manager.get_map('ModelSEED', 'ModelSEED', 'Central_Metabolism_JE')
uids = em.add_uid_to_reaction_metabolites()
def move_to_compartment(cmp_id, em):
    em.add_uid_to_reaction_metabolites()
    node_uid_cmp = {}
    node_uid_id = {}
    for node_uid in em.escher_graph['nodes']:
        n = em.escher_graph['nodes'][node_uid]
        if n['node_type'] == 'metabolite':
            if not 'compartment' in n:
                node_uid_cmp[node_uid] = cmp_id
                n['bigg_id'] += '_' + cmp_id
                node_uid_id[node_uid] = n['bigg_id']
            else:
                node_uid_id[node_uid] = n['bigg_id']
    add_compartment(em, node_uid_cmp)
    for rxn_uid in em.escher_graph['reactions']:
        rnode = em.escher_graph['reactions'][rxn_uid]
        for o in rnode['metabolites']:
            o['bigg_id'] = node_uid_id[o['node_uid']]
            
        rnode['bigg_id'] += '_' + cmp_id
    return node_uid_cmp
cmp = move_to_compartment('z', em)

!!! cpd00011
!!! cpd00011


In [ ]:
ls 

In [61]:
em = None
with open('/Users/fliu/workspace/jupyter/data/escher/maps/ModelSEED/ModelSEED.Central_Metabolism_JE.json', 'r') as fh:
    em = json.loads(fh.read())

In [62]:
for uid in em[1]['nodes']:
    n = em[1]['nodes'][uid]
    if n['node_type'] == 'metabolite':
        if n['bigg_id'].endswith('_e'):
            n['compartment'] = 'e'

In [63]:
with open('/Users/fliu/workspace/jupyter/data/escher/maps/ModelSEED/ModelSEED.Central_Metabolism_JE.json', 'w') as fh:
    fh.write(json.dumps(em))

In [65]:
build_data = {"maps":["iMM904;c;c;ModelSEED.Central_Metabolism_JE"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
print(map_assembly)

[{'map_id': 'Central_Metabolism_JE', 'cmp_target': ['c'], 'cmp_sbml': ['c'], 'sbml_id': 'iMM904'}]


In [22]:
build_data = {"maps":["iMM904;c;c;ModelSEED.Tryptophan Biosynthesis"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
print(map_assembly)

[{'map_id': 'Tryptophan Biosynthesis', 'cmp_target': ['c'], 'cmp_sbml': ['c'], 'sbml_id': 'iMM904'}]


In [178]:
escher_factory = EscherFactoryApi(escher_manager)
escher_factory.cpd_mapping = MODEL_CPD_MAPPING
escher_factory.rxn_mapping = {}
#escher_factory.bios_cache = bios.model_data
escher_factory.cpd_mapping['core'] = wut_map
escher_factory.rxn_mapping['core'] = wut_rxn_map

In [97]:
core_model = None
with open('/Users/fliu/workspace/jupyter/data/escher/models/ModelSEED/Core.json', 'r') as fh:
    core_model = json.loads(fh.read())

In [155]:
wut_map = {}
for o in core_model['metabolites']:
    #print(o['id'], o['compartment'])
    if not o['compartment'] in wut_map:
        wut_map[o['compartment']] = {}
    wut_map[o['compartment']][o['id']] = [o['id']]

In [177]:
wut_rxn_map = {}
for o in core_model['reactions']:
    wut_rxn_map[o['id']] = [o['id']]

In [141]:
#escher_factory.get_cpd_mapping('iMM904','c')
#{'cpd00201': ['M_10fthf_c'],

In [79]:
build_data = {"maps":["core;c;c;ModelSEED.Central_Metabolism_JE"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
print(map_assembly)

[{'map_id': 'Central_Metabolism_JE', 'cmp_target': ['c'], 'cmp_sbml': ['c'], 'sbml_id': 'core'}]


In [184]:
MODEL_CPD_MAPPING = {}
with open('../data/cpd_mapping_cache4.json', 'r') as f:
    MODEL_CPD_MAPPING = json.loads(f.read())
MODEL_RXN_MAPPING = {}
with open('../data/rxn_mapping_cache4.json', 'r') as f:
    MODEL_RXN_MAPPING = json.loads(f.read())

In [ ]:
MODEL_CPD_MAPPING['core'] = wut_map

In [179]:
em_list = []
for ma in map_assembly:
    escher_model_id = 'ModelSEED'
    escher_map_id = ma['map_id']
    sbml_id = ma['sbml_id']
    target_cmp = ma['cmp_target'][0]
    cmp_sbml = ma['cmp_sbml'][0]
    cpd_mapping = escher_factory.get_cpd_mapping('core','c')
    rxn_mapping = escher_factory.get_rxn_mapping('core')
    #em = self.escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)

In [166]:
escher_factory.cpd_match_exclude

set()

In [169]:
f = escher_factory.lambda_get_model_reaction('core')

In [170]:
f

<function EscherFactoryApi.lambda_get_model_reaction.<locals>.<lambda>(x)>

In [181]:
em = escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)
em.add_uid_to_reaction_metabolites()
a = move_to_compartment(target_cmp, em)
def remap_map_compounds(em, bigg_to_seed, id_function = lambda x : (x, 0)):
    #print(bigg_to_seed)
    map_compound_remap = {}
    unmaped = set()
    node_uid_cmp = {}
    for map_uid in em.escher_graph['nodes']:
        node = em.escher_graph['nodes'][map_uid]
        if node['node_type'] == 'metabolite':
            node_id = node['bigg_id']
            #print(node_id, node)
            bigg_id, cmp = id_function(node_id)
            #print(node_id, bigg_id, cmp)
            #bigg_id = node_id[:-2]
            #cmp = node_id[-1:]
            
            #print(node_id, bigg_id, cmp)
            if bigg_id in bigg_to_seed:
                node_uid_cmp[map_uid] = cmp
                map_compound_remap[node_id] = set(bigg_to_seed[bigg_id])
                #print(map_uid, bigg_id, cmp, bigg_to_seed[bigg_id])
            else:
                unmaped.add(node_id)
    return map_compound_remap, unmaped, node_uid_cmp
map_compound_remap, unmaped_cpd, node_uid_cmp = remap_map_compounds(em, cpd_mapping, lambda x : x.split('_'))
logger.warning('unmaped_cpd: %d', len(unmaped_cpd))
map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_mapping, map_compound_remap, escher_factory.cpd_match_exclude,
                                                              escher_factory.lambda_get_model_reaction(sbml_id),
                                                              lambda x : x, lambda x : x[:-2])
logger.warning('unmaped_rxn: %d', len(unmaped_rxn))

print(sbml_id, map_reaction_remap)

cpd_remap = {}
rxn_remap = {}
for k in map_compound_remap:
    cpd_remap[k] = list(map_compound_remap[k])[0] + '@' + sbml_id
for k in map_reaction_remap:
    rxn_remap[k] = list(map_reaction_remap[k])[0] + '@' + sbml_id
#print(sbml_id, rxn_remap)
em.swap_ids(cpd_remap, rxn_remap)
em.delete_reactions(unmaped_rxn)
em.delete_metabolites(unmaped_cpd)

!!! cpd00011
!!! cpd00011
core {}


In [182]:
em.display_in_notebook()

In [ ]:
def translate_to_model(self, escher_model_id, escher_map_id, sbml_id, target_cmp, cpd_mapping, rxn_mapping):
        em = self.escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)
        em.add_uid_to_reaction_metabolites()

        move_to_compartment(target_cmp, em)

        map_compound_remap, unmaped_cpd, node_uid_cmp = remap_map_compounds(em, 
                                                                            cpd_mapping, lambda x : x.split('_'))

        logger.warning('unmaped_cpd: %d', len(unmaped_cpd))

        map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_mapping, map_compound_remap, self.cpd_match_exclude,
                                                              self.lambda_get_model_reaction(sbml_id),
                                                              lambda x : x, lambda x : x[:-2])

        logger.warning('unmaped_rxn: %d', len(unmaped_rxn))

        print(sbml_id, map_reaction_remap)
        
        cpd_remap = {}
        rxn_remap = {}
        for k in map_compound_remap:
            cpd_remap[k] = list(map_compound_remap[k])[0] + '@' + sbml_id
        for k in map_reaction_remap:
            rxn_remap[k] = list(map_reaction_remap[k])[0] + '@' + sbml_id
        #print(sbml_id, rxn_remap)
        em.swap_ids(cpd_remap, rxn_remap)
        em.delete_reactions(unmaped_rxn)
        em.delete_metabolites(unmaped_cpd)
        return em